In [20]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import imodelsx.process_results
import sys
import datasets
import numpy as np
from copy import deepcopy
import clin.eval.eval
import clin.eval.med_status
import clin.llm
import clin.parse
from collections import defaultdict
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'

sys.path.append('../experiments/')
results_dir = '../results/'

r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
r = r[r.dataset_name == 'medication_status']

# get data for eval
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
n = len(df)

r = clin.eval.med_status.add_status_eval(r, df)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 17/17 [00:00<00:00, 1907.42it/s]


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# print errs for med extraction
# mets = clin.eval.calculate_metrics(row['dict_ov_pv_ev'], df, verbose=True)

# print errors for status extraction
# accs_cond, f1s_macro_cond = clin.eval.eval_medication_status([med_status_dicts_list[-1]], df, verbose=True)

## Interpret an example

In [ ]:
# single row to investigate
row = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')].iloc[0]

In [ ]:
i = 11
ev = row['dict_evidence_ov_pv_ev'][i]
med_status_dict = row['dict_sv'][i]
gt = df.iloc[i]
print(gt)
print(med_status_dict)
print('\n' + gt['snippet'])

for k in ev:
    print(k)
    print('\t', ev[k])
    print()